In [ ]:
import cv2     
import math   
import matplotlib.pyplot as plt    
%matplotlib inline
import pandas as pd
from keras.preprocessing import image  
import numpy as np    
from keras.utils import np_utils
from skimage.transform import resize  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
count = 0
videoFile = "/content/drive/MyDrive/tom and jerry/training/Tom and jerry.mp4"
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="/content/drive/MyDrive/tom and jerry/training/frames/frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [ ]:
img = plt.imread('/content/drive/MyDrive/tom and jerry/training/frames/frame0.jpg')   # reading image using its name
plt.imshow(img)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/tom and jerry/training/mapping (1).csv')     # reading the csv file
data.head()      # printing first five rows of the file

In [ ]:
X = [ ]     # creating an empty array
for img_name in data.Image_ID:
    img = plt.imread('/content/drive/MyDrive/tom and jerry/training/frames/' + img_name)
    X.append(img)  # storing each image in array X
X = np.array(X)    # converting list to array

In [ ]:
y = data.Class
dummy_y = np_utils.to_categorical(y)    # one hot encoding Classes

In [ ]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

In [ ]:
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)      # preprocessing the input data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.2, random_state=42)    # preparing the validation set

In [ ]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    # include_top=False to remove the top layer

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

((238, 7, 7, 512), (60, 7, 7, 512))

In [ ]:
X_train = X_train.reshape(238, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(60, 7*7*512)

In [ ]:
train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()

In [ ]:
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid'))   # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=512, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=256, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(4, activation='softmax'))            # output layer

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 1

In [ ]:
# ii. Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(train, y_train, epochs=100, validation_data=(X_valid, y_valid))

Epoch 1/100
8/8 [==============================] - 8s 75ms/step - loss: 1.5815 - accuracy: 0.3167 - val_loss: 1.2365 - val_accuracy: 0.3500
Epoch 2/100
8/8 [==============================] - 0s 12ms/step - loss: 1.2840 - accuracy: 0.3333 - val_loss: 1.1648 - val_accuracy: 0.3500
Epoch 3/100
8/8 [==============================] - 0s 12ms/step - loss: 1.2722 - accuracy: 0.4510 - val_loss: 1.1424 - val_accuracy: 0.5000
Epoch 4/100
8/8 [==============================] - 0s 12ms/step - loss: 1.2515 - accuracy: 0.4448 - val_loss: 1.0752 - val_accuracy: 0.5833
Epoch 5/100
8/8 [==============================] - 0s 12ms/step - loss: 1.1606 - accuracy: 0.4814 - val_loss: 0.9864 - val_accuracy: 0.6333
Epoch 6/100
8/8 [==============================] - 0s 12ms/step - loss: 1.0589 - accuracy: 0.5817 - val_loss: 0.8752 - val_accuracy: 0.6667
Epoch 7/100
8/8 [==============================] - 0s 12ms/step - loss: 0.8469 - accuracy: 0.7404 - val_loss: 0.7330 - val_accuracy: 0.7667
Epoch 8/100
8/8 [===

In [ ]:
count = 0
videoFile = "/content/drive/MyDrive/tom and jerry/testing/Tom and Jerry 3.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="/content/drive/MyDrive/tom and jerry/testing/frames/test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/tom and jerry/testing/test (1).csv')

In [ ]:
test_image = []
for img_name in test.Image_ID:
    img = plt.imread('/content/drive/MyDrive/tom and jerry/testing/frames/' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [ ]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

In [ ]:
# preprocessing the images
test_image = preprocess_input(test_image)

# extracting features from the images using pretrained model
test_image = base_model.predict(test_image)

# converting the images to 1-D form
test_image = test_image.reshape(186, 7*7*512)

# zero centered images
test_image = test_image/test_image.max()

In [ ]:
predictions = model.predict_classes(test_image)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
print("The screen time of JERRY is", predictions[predictions==1].shape[0], "seconds")
print("The screen time of TOM is", predictions[predictions==2].shape[0], "seconds")

The screen time of JERRY is 3 seconds
The screen time of TOM is 102 seconds
